In [1]:
using JuMP, Ipopt, Plots, DataFrames
gr()

Plots.GRBackend()

# AC Power Flow algorithm

## AC Power Flow analysis
An alternating current power-flow model is a model used in electrical engineering to analyze power grids. It provides a nonlinear system which describes the energy flow through each transmission line. The goal of a power-flow study is to obtain complete voltages angle and magnitude information for each bus in a power system for specified load and generator real power and voltage conditions.  

Once this information is known, real and reactive power flow on each branch as well as generator reactive power output can be analytically determined. Power-flow or load-flow studies are important for planning future expansion of power systems as well as in determining the best operation of existing systems. 

## Problem description
- Consider a network with 5 buses that forms a cycle (i.e., the lines are (1,2), (2,3), (3,4),(4,5) and (5,1)).
- Assume that Bus 1 is a slack, Bus 2 is PV (generator) and Buses 3-5 are PQ (loads).
- Assume that the resistance and reactance of each line are both equal to 1.

Repeat the following lines 1-3 several times (say 100 times):
- 1: Generate a random vector V such that all voltage magnitudes are somehow close to 1 and all voltage angles are close to 0, and that the phase at the slack bus is zero.
- 2: Based on the random vector of voltages V, compute the loads at the PQ buses, the P and |V| values at the PV buses, and |V| at the slack bus. (constraints/input data needed to run the model)
- 3: Use the measurement data to solve the power flow problem in two ways: (1) Newton's Mehtod, (2) JuMP 
- 4: Declare a success if the obtained solution matches the original random state V. Compute how many times each of the above two methods was successful for different values of voltage angles (sensitivity analysis).

#### NOTE: The problem does not always have a solution, especially at large angles

## Mathematical Formulation
The Power flow problem requires us to determine voltages angle and magnitude information for each bus in a power system for specified load and generator real power and voltage conditions:
- For generator buses (also called PV) - we are given Pᵢ & |Vᵢ| -> need to solve for θᵢ & Qᵢ
- For load buses (also called PQ) - we are given Pᵢ & Qᵢ -> need to solve for θᵢ & |Vᵢ|
- For the slack bus (used to balance other buses) - we are given |Vᵢ| & θᵢ -> need to solve for Pᵢ & Qᵢ

Therefore our problem can be stated as:
- Known parameters: V₁, θ₁, P₂, V₂, P₃, Q₃, P₄, Q₄, P₅, Q₅
- List of unknowns: P₁, Q₁, Q₂, θ₂, V₃, θ₃, V₄, θ₄, V₅, θ₅
- Need to write equations P₂, P₃, Q₃, P₄, Q₄, P₅, Q₅

Where:
- Pᵢ = ∑|Vᵢ||Vⱼ|[Gᵢⱼ cos(θᵢⱼ) + Bᵢⱼ sin(θᵢⱼ)] 
- Qᵢ = ∑|Vᵢ||Vⱼ|[Gᵢⱼ cos(θᵢⱼ) - Bᵢⱼ sin(θᵢⱼ)]

And:
- θᵢⱼ = θᵢ - θⱼ
- Gᵢⱼ = Real(Yᵢⱼ) # admittance matrix
- Bᵢⱼ = Imaginary(Yᵢⱼ)
- Yᵢⱼ is the inverse of the impedance matrix which depends on the resistance and reactance of power lines

I will use the equations in terms of vector x to avoid confusion
x = [θ₂ θ₃ θ₄ θ₅ V₃ V₄ V₅]

## Newton's Method

We will apply a multivariable Newton's method to solve the Non-linear system of power equations according to the following algorithm:

x = [θ₂, θ₃, θ₄, θ₅, V₃, V₄, V₅]

fx = [P₂(x)-P0₂ , P₃(x)-P0₃ , P₄(x)-P0₄ , P₅(x)-P0₅ , Q₃(x)-Q0₃ , Q₄(x)-Q0₄ , Q₅(x)-Q0₅]

J = δfxᵢ / δxⱼ

while ||fx(xⁿ)|| < ϵ

    xⁿ⁺¹ = xⁿ - J(xⁿ)⁻¹f(xⁿ)
    xⁿ = xⁿ⁺¹
    
end

### Constructing the Admittance Matrix

In [2]:
# Since we are not changing resistance or reactance, the admittance matrix will always be constant

Zij = zeros(5,5)
Zij = complex(Zij)
Zij[1,2] = 1+1im; Zij[2,1] = 1+1im;
Zij[2,3] = 1+1im; Zij[3,2] = 1+1im;
Zij[3,4] = 1+1im; Zij[4,3] = 1+1im;
Zij[4,5] = 1+1im; Zij[5,4] = 1+1im;
Zij[5,1] = 1+1im; Zij[1,5] = 1+1im;
Y = zeros(5,5);
Y = complex(Y)
for i=1:5, j=1:5
    if i != j
        if Zij[i,j] != 0
            Y[i,j] = -Zij[i,j]^-1;
        else
            Y[i,j] = 0;
        end
    end
end
for i = 1:5
    Y[i,i] = -sum(Y[i,:]);
end
Y

5×5 Array{Complex{Float64},2}:
  1.0-1.0im  -0.5+0.5im   0.0+0.0im   0.0+0.0im  -0.5+0.5im
 -0.5+0.5im   1.0-1.0im  -0.5+0.5im   0.0+0.0im   0.0+0.0im
  0.0+0.0im  -0.5+0.5im   1.0-1.0im  -0.5+0.5im   0.0+0.0im
  0.0+0.0im   0.0+0.0im  -0.5+0.5im   1.0-1.0im  -0.5+0.5im
 -0.5+0.5im   0.0+0.0im   0.0+0.0im  -0.5+0.5im   1.0-1.0im

### Define a function to evaluate fx (active and reactive power)

In [3]:
function feval(Y, V; f0 = zeros(7))

function f(x::Vector)
fx = zeros(7)
θ₁ = 0; #slack reference bus
fx[1] = V[2]*V[1]*(real(Y[2,1])*cos(x[1]-θ₁)+imag(Y[2,1])*sin(x[1]-θ₁))+ V[2]*V[2]*real(Y[2,2])+
        V[2]*x[5]*(real(Y[2,3])*cos(x[1]-x[2])+imag(Y[2,3])*sin(x[1]-x[2])) - f0[1]
fx[2] = x[5]*V[2]*(real(Y[3,2])*cos(x[2]-x[1])+imag(Y[3,2])*sin(x[2]-x[1]))+ x[5]*x[5]*real(Y[3,3])+
        x[5]*x[6]*(real(Y[3,4])*cos(x[2]-x[3])+imag(Y[3,4])*sin(x[2]-x[3])) - f0[2]
fx[3] = x[6]*x[5]*(real(Y[4,3])*cos(x[3]-x[2])+imag(Y[4,3])*sin(x[3]-x[2]))+ x[6]*x[6]*real(Y[4,4])+
        x[6]*x[7]*(real(Y[4,5])*cos(x[3]-x[4])+imag(Y[4,5])*sin(x[3]-x[4])) - f0[3]
fx[4] = x[7]*x[6]*(real(Y[5,4])*cos(x[4]-x[3])+imag(Y[5,4])*sin(x[4]-x[3]))+ x[7]*x[7]*real(Y[5,5])+
        x[7]*V[1]*(real(Y[5,1])*cos(x[4]-θ₁)+imag(Y[5,4])*sin(x[4]-θ₁)) - f0[4]
fx[5] = x[5]*V[2]*(real(Y[3,2])*sin(x[2]-x[1])-imag(Y[3,2])*cos(x[2]-x[1])) -x[5]*x[5]*imag(Y[3,3])+
        x[5]*x[6]*(real(Y[3,4])*sin(x[2]-x[3])-imag(Y[3,4])*cos(x[2]-x[3])) - f0[5]
fx[6] = x[6]*x[5]*(real(Y[4,3])*sin(x[3]-x[2])-imag(Y[4,3])*cos(x[3]-x[2]))-x[6]*x[6]*imag(Y[4,4])+
        x[6]*x[7]*(real(Y[4,5])*sin(x[3]-x[4])-imag(Y[4,5])*cos(x[3]-x[4])) - f0[6]
fx[7] = x[7]*x[6]*(real(Y[5,4])*sin(x[4]-x[3])-imag(Y[5,4])*cos(x[4]-x[3]))-x[7]*x[7]*imag(Y[5,5])+
        x[7]*V[1]*(real(Y[5,1])*sin(x[4]-θ₁)-imag(Y[5,1])*cos(x[4]-θ₁)) - f0[7]
return fx
end
    
end

feval (generic function with 1 method)

### Define a function to evaluate Jacobian

#### I did the analytical Jacobian, tried to use autodiff but i couldn't make it work

In [4]:
function Jeval(Y, V)

function J(x::Vector)
θ₁ = 0 #slack reference bus
df1x1 = V[2]*V[1]*(-real(Y[2,1])*sin(x[1]-θ₁)+imag(Y[2,1])*cos(x[1]-θ₁))+
            V[2]*x[5]*(-real(Y[2,3])*sin(x[1]-x[2])+imag(Y[2,3])*cos(x[1]-x[2]))
df1x2 = V[2]*x[5]*(real(Y[2,3])*sin(x[1]-x[2])-imag(Y[2,3])*cos(x[1]-x[2]))
df1x3 = 0
df1x4 = 0
df1x5 = V[2]*(real(Y[2,3])*cos(x[1]-x[2])+imag(Y[2,3])*sin(x[1]-x[2]))
df1x6 = 0
df1x7 = 0
df2x1 = x[5]*V[2]*(real(Y[3,2])*sin(x[2]-x[1])-imag(Y[3,2])*cos(x[2]-x[1]))
df2x2 = x[5]*V[2]*(-real(Y[3,2])*sin(x[2]-x[1])+imag(Y[3,2])*cos(x[2]-x[1]))+
            x[5]*x[6]*(-real(Y[3,4])*sin(x[2]-x[3])+imag(Y[3,4])*cos(x[2]-x[3]))
df2x3 = x[5]*x[6]*(real(Y[3,4])*sin(x[2]-x[3])-imag(Y[3,4])*cos(x[2]-x[3]))
df2x4 = 0;
df2x5 = V[2]*(real(Y[3,2])*cos(x[2]-x[1])+imag(Y[3,2])*sin(x[2]-x[1]))+
            2*x[5]*real(Y[3,3])+x[6]*(real(Y[3,4])*cos(x[2]-x[3])+imag(Y[3,4])*sin(x[2]-x[3]))
df2x6 = x[5]*(real(Y[3,4])*cos(x[2]-x[3])+imag(Y[3,4])*sin(x[2]-x[3]))
df2x7 = 0
df3x1 = 0
df3x2 = x[6]*x[5]*(real(Y[4,3])*sin(x[3]-x[2])-imag(Y[4,3])*cos(x[3]-x[2]))
df3x3 = x[6]*x[5]*(-real(Y[4,3])*sin(x[3]-x[2])+imag(Y[4,3])*cos(x[3]-x[2]))+
            x[6]*x[7]*(-real(Y[4,5])*sin(x[3]-x[4])+imag(Y[4,5])*cos(x[3]-x[4]))
df3x4 = x[6]*x[7]*(real(Y[4,5])*sin(x[3]-x[4])-imag(Y[4,5])*cos(x[3]-x[4]))
df3x5 = x[6]*(real(Y[4,3])*cos(x[3]-x[2])+imag(Y[4,3])*sin(x[3]-x[2]))
df3x6 = x[5]*(real(Y[4,3])*cos(x[3]-x[2])+imag(Y[4,3])*sin(x[3]-x[2]))+
            2*x[6]*real(Y[4,4])+x[7]*(real(Y[4,5])*cos(x[3]-x[4])+imag(Y[4,5])*sin(x[3]-x[4]))
df3x7 = x[6]*(real(Y[4,5])*cos(x[3]-x[4])+imag(Y[4,5])*sin(x[3]-x[4]))
df4x1 = 0
df4x2 = 0
df4x3 = x[7]*x[6]*(real(Y[5,4])*sin(x[4]-x[3])-imag(Y[5,4])*cos(x[4]-x[3]))
df4x4 = x[7]*x[6]*(-real(Y[5,4])*sin(x[4]-x[3])+imag(Y[5,4])*cos(x[4]-x[3]))+
            x[7]*V[1]*(-real(Y[5,1])*sin(x[4]-θ₁)+imag(Y[5,1])*cos(x[4]-θ₁))
df4x5 = 0
df4x6 = x[7]*(real(Y[5,4])*cos(x[4]-x[3])+imag(Y[5,4])*sin(x[4]-x[3]))
df4x7 = x[6]*(real(Y[5,4])*cos(x[4]-x[3])+imag(Y[5,4])*sin(x[4]-x[3]))+
            2*x[7]*real(Y[5,5])+V[1]*(real(Y[5,1])*cos(x[4]-θ₁)+imag(Y[5,1])*sin(x[4]-θ₁))
df5x1 = x[5]*V[2]*(-real(Y[3,2])*cos(x[2]-x[1])-imag(Y[3,2])*sin(x[2]-x[1]))
df5x2 = x[5]*V[2]*(real(Y[3,2])*cos(x[2]-x[1])+imag(Y[3,2])*sin(x[2]-x[1]))+
                        x[5]*x[6]*(real(Y[3,4])*cos(x[2]-x[3])+imag(Y[3,4])*sin(x[2]-x[3]))
df5x3 = x[5]*x[6]*(-real(Y[3,4])*cos(x[2]-x[3])-imag(Y[3,4])*sin(x[2]-x[3]))
df5x4 = 0;
df5x5 = V[2]*(real(Y[3,2])*sin(x[2]-x[1])-imag(Y[3,2])*cos(x[2]-x[1]))-
            2*x[5]*imag(Y[3,3])+x[6]*(real(Y[3,4])*sin(x[2]-x[3])-imag(Y[3,4])*cos(x[2]-x[3]))
df5x6 = x[5]*(real(Y[3,4])*sin(x[2]-x[3])-imag(Y[3,4])*cos(x[2]-x[3]))
df5x7 = 0
df6x1 = 0
df6x2 = x[6]*x[5]*(-real(Y[4,3])*cos(x[3]-x[2])-imag(Y[4,3])*sin(x[3]-x[2]))
df6x3 = x[6]*x[5]*(real(Y[4,3])*cos(x[3]-x[2])+imag(Y[4,3])*sin(x[3]-x[2]))+
            x[6]*x[7]*(real(Y[4,5])*cos(x[3]-x[4])+imag(Y[4,5])*sin(x[3]-x[4]))
df6x4 = x[6]*x[7]*(-real(Y[4,5])*cos(x[3]-x[4])-imag(Y[4,5])*sin(x[3]-x[4]))
df6x5 = x[6]*(real(Y[4,3])*sin(x[3]-x[2])-imag(Y[4,3])*cos(x[3]-x[2]))
df6x6 = x[5]*(real(Y[4,3])*sin(x[3]-x[2])-imag(Y[4,3])*cos(x[3]-x[2]))-
            2*x[6]*imag(Y[4,4])+x[7]*(real(Y[4,5])*sin(x[3]-x[4])-imag(Y[4,5])*cos(x[3]-x[4]))
df6x7 = x[6]*(real(Y[4,5])*sin(x[3]-x[4])-imag(Y[4,5])*cos(x[3]-x[4]))
df7x1 = 0
df7x2 = 0
df7x3 = x[7]*x[6]*(-real(Y[5,4])*cos(x[4]-x[3])-imag(Y[5,4])*sin(x[4]-x[3]))
df7x4 = x[7]*x[6]*(real(Y[5,4])*cos(x[4]-x[3])+imag(Y[5,4])*sin(x[4]-x[3]))+
            x[7]*V[1]*(real(Y[5,1])*cos(x[4]-θ₁)+imag(Y[5,1])*sin(x[4]-θ₁))
df7x5 = 0
df7x6 = x[7]*(real(Y[5,4])*sin(x[4]-x[3])-imag(Y[5,4])*cos(x[4]-x[3]))
df7x7 = x[6]*(real(Y[5,4])*sin(x[4]-x[3])-imag(Y[5,4])*cos(x[4]-x[3]))-
            2*x[7]*imag(Y[5,5])+V[1]*(real(Y[5,1])*sin(x[4]-θ₁)-imag(Y[5,1])*cos(x[4]-θ₁))
Jx = [
    df1x1 df1x2 df1x3 df1x4 df1x5 df1x6 df1x7;
    df2x1 df2x2 df2x3 df2x4 df2x5 df2x6 df2x7;
    df3x1 df3x2 df3x3 df3x4 df3x5 df3x6 df3x7;
    df4x1 df4x2 df4x3 df4x4 df4x5 df4x6 df4x7;
    df5x1 df5x2 df5x3 df5x4 df5x5 df5x6 df5x7;
    df6x1 df6x2 df6x3 df6x4 df6x5 df6x6 df6x7;
    df7x1 df7x2 df7x3 df7x4 df7x5 df7x6 df7x7;
]
return Jx
                                
end

end

Jeval (generic function with 1 method)

### Compute the constraints at the PQ & PV buses and the slack bus [P0 & Q0]

In [5]:
V = 0.9+0.2*rand(5,1) # random number between 0.9-1.1
θ = (-2+2*2*randn(5,1))*(π/180) # random angle between -2º to 2º
V[1] = 1 # slack bus voltage magnitude (reference)
θ[1] = 0 # slack bus voltage angle (reference)

0

In [6]:
f0 = feval(Y,V) # generates a function to evaluate power flows in terms of x
constraints = f0(x_true) # computes constraints based on initial vector x0

LoadError: UndefVarError: x_true not defined

### Solution with Newton's algorithm

In [7]:
function PF_newton(feval, Jeval, V, θ, Y; x0 = [0,0,0,0,1,1,1],tol = 1e-9, xtol = 1e-4, max_iter = 15)

x_true = [θ[2]; θ[3]; θ[4]; θ[5]; V[3]; V[4]; V[5]] # original x vector (angles in radians)
    
f0 = feval(Y,V) # generates a function to evaluate power flows in terms of x
constraints = f0(x_true) # computes constraints based on initial vector x0

# compute new fx function incorporating constraints and Jacobian
fx = feval(Y,V, f0 = constraints)
Jx = Jeval(Y,V)

#initialize algorithm
x = x0

for i = 1:max_iter
    xn = x - Jx(x)\fx(x)
    x = xn

# Was a solution found?
if sum(fx(x).^2) < tol
    break
end
    
end

# Is it the correct soultion? It is possible to have more than 1 local solution
if sum((x-x_true).^2) < xtol
    success_newt = true
else
    success_newt = false
end

    return success_newt, x
end

PF_newton (generic function with 1 method)

In [8]:
(success_newt, x_newt) = PF_newton(feval, Jeval, V, θ, Y)

(true,[-0.0612468,-0.00891996,0.0209283,0.0493972,1.07342,0.972637,0.919865])

## Solving with JuMP

In [9]:
function PF_JuMP(feval, V, θ, Y; xtol = 1e-4)

x_true = [θ[2]; θ[3]; θ[4]; θ[5]; V[3]; V[4]; V[5]] # original x vector (angles in radians)    

# JuMP does not recognize real and imaginary functions so I have to define the Real and Imaginary Parts of Y
YR = real(Y); YI = imag(Y); θ₁ = 0;
    
f0 = feval(Y,V) # generates a function to evaluate power flows in terms of x
constraints = f0(x_true) # computes constraints based on initial vector x0

# Initially Ipopt Solver
m = Model(solver=IpoptSolver(print_level=0))
# Define our x variable
@variable(m, x[1:7])

# Set initial guess
for i in 1:4
    setvalue(x[i], 0.0)
end

for i in 5:7
    setvalue(x[i], 1.0)
end


@NLobjective(m, Max, 10) #doesn't matter we can write constant

# Active Power & Reactive Power non-linear constraints
# I must define the constraints explicitly since JuMP does not support vectorized form in constraints ...
# If not I would have to define an expression and it becomes too much effort, but it would make it cleaner

@NLconstraints(m, begin
        
    V[2]*V[1]*((YR[2,1])*cos(x[1]-θ₁)+(YI[2,1])*sin(x[1]-θ₁))+ V[2]*V[2]*(YR[2,2])+ 
        V[2]*x[5]*((YR[2,3])*cos(x[1]-x[2])+(YI[2,3])*sin(x[1]-x[2])) == constraints[1]
        
    x[5]*V[2]*((YR[3,2])*cos(x[2]-x[1])+(YI[3,2])*sin(x[2]-x[1]))+ x[5]*x[5]*(YR[3,3])+ 
        x[5]*x[6]*((YR[3,4])*cos(x[2]-x[3])+(YI[3,4])*sin(x[2]-x[3])) == constraints[2]
        
    x[6]*x[5]*((YR[4,3])*cos(x[3]-x[2])+(YI[4,3])*sin(x[3]-x[2]))+ x[6]*x[6]*(YR[4,4])+ 
        x[6]*x[7]*((YR[4,5])*cos(x[3]-x[4])+(YI[4,5])*sin(x[3]-x[4])) == constraints[3]
    
    x[7]*x[6]*((YR[5,4])*cos(x[4]-x[3])+(YI[5,4])*sin(x[4]-x[3]))+ x[7]*x[7]*(YR[5,5])+ 
        x[7]*V[1]*((YR[5,1])*cos(x[4]-θ₁)+(YI[5,4])*sin(x[4]-θ₁)) == constraints[4]
        
    x[5]*V[2]*((YR[3,2])*sin(x[2]-x[1])-(YI[3,2])*cos(x[2]-x[1]))- x[5]*x[5]*(YI[3,3])+
        x[5]*x[6]*((YR[3,4])*sin(x[2]-x[3])-(YI[3,4])*cos(x[2]-x[3])) == constraints[5]
    
    x[6]*x[5]*((YR[4,3])*sin(x[3]-x[2])-(YI[4,3])*cos(x[3]-x[2]))- x[6]*x[6]*(YI[4,4])+
        x[6]*x[7]*((YR[4,5])*sin(x[3]-x[4])-(YI[4,5])*cos(x[3]-x[4])) == constraints[6]
        
    x[7]*x[6]*((YR[5,4])*sin(x[4]-x[3])-(YI[5,4])*cos(x[4]-x[3]))- x[7]*x[7]*(YI[5,5])+
        x[7]*V[1]*((YR[5,1])*sin(x[4]-θ₁)-(YI[5,1])*cos(x[4]-θ₁)) == constraints[7]
        
end)

solve(m; suppress_warnings=true)
x_JuMP = getvalue(x);

# Is it the correct soultion? It is possible to have more than 1 local solution
if sum((x_JuMP-x_true).^2) < xtol
    success_JuMP = true
else
    success_JuMP = false
end
    return success_JuMP, x_JuMP
end

PF_JuMP (generic function with 1 method)

In [10]:
(success_JuMP, x_JuMP) = PF_JuMP(feval, V, θ, Y)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



(true,[-0.061247,-0.00892008,0.0209282,0.0493972,1.07342,0.972637,0.919865])

## Results for one iteration

In [11]:
Results = DataFrame(x_true = x_true, x_newton = x_newt, x_JuMP = x_JuMP)

LoadError: UndefVarError: x_true not defined

In [12]:
@time PF_JuMP(feval, V, θ, Y)

  0.003689 seconds (3.43 k allocations: 252.500 KB)


(true,[-0.061247,-0.00892008,0.0209282,0.0493972,1.07342,0.972637,0.919865])

In [13]:
@time PF_newton(feval,Jeval, V, θ, Y) # although it took me ages to do the Jacobian manually

  0.000183 seconds (525 allocations: 80.766 KB)


(true,[-0.0612468,-0.00891996,0.0209283,0.0493972,1.07342,0.972637,0.919865])

## Sensitivity Analysis for voltage angle range

In [14]:
for i = 5:10:45
    
θ_val = i;
s_newt = 0;
s_JuMP = 0;

# perform 100 iterations for each angle range
for k=1:100
    V = 0.9+0.2*rand(5,1) # random number between 0.9-1.1
    θ = (-θ_val+2*θ_val*randn(5,1))*(π/180) # random angle between -5º to 5º
    V[1] = 1 # slack bus voltage magnitude (reference)
    θ[1] = 0 # slack bus voltage angle (reference)
    
    (success_JuMP, x_JuMP) = PF_JuMP(feval, V, θ, Y)
    (success_newt, x_newt) = PF_newton(feval,Jeval, V, θ, Y)
    
    if success_JuMP == true
        s_JuMP = s_JuMP +1;
    end
    if success_newt == true
        s_newt = s_newt +1;
    end
end
    
println("With θ = -$θ_val degrees to $θ_val degrees")
println("Newton success rate is $s_newt")
println("JuMP success rate is $s_JuMP")
println("__________________________________________")    
end
    

With θ = -5 degrees to 5 degrees
Newton success rate is 100
JuMP success rate is 100
__________________________________________
With θ = -15 degrees to 15 degrees
Newton success rate is 46
JuMP success rate is 32
__________________________________________
With θ = -25 degrees to 25 degrees
Newton success rate is 19
JuMP success rate is 12
__________________________________________
With θ = -35 degrees to 35 degrees
Newton success rate is 13
JuMP success rate is 5
__________________________________________
With θ = -45 degrees to 45 degrees
Newton success rate is 5
JuMP success rate is 0
__________________________________________


In [34]:
# We can also plot the results

angle_amplitude = Int64[]
ss_JuMP = Int64[]
ss_newt = Int64[]

for i = 2:2:90
    
θ_val = i;
s_newt = 0;
s_JuMP = 0;

# perform 100 iterations for each angle range
for k=1:100
    V = 0.9+0.2*rand(5,1) # random number between 0.9-1.1
    θ = (-θ_val+2*θ_val*randn(5,1))*(π/180) # random angle between -5º to 5º
    V[1] = 1 # slack bus voltage magnitude (reference)
    θ[1] = 0 # slack bus voltage angle (reference)
    
    (success_JuMP, x_JuMP) = PF_JuMP(feval, V, θ, Y)
    (success_newt, x_newt) = PF_newton(feval,Jeval, V, θ, Y)
    
    if success_JuMP == true
        s_JuMP = s_JuMP +1
    end
    if success_newt == true
        s_newt = s_newt +1
    end
end
    
push!(angle_amplitude, i)    
push!(ss_JuMP, s_JuMP)
push!(ss_newt, s_newt)
       
end

In [54]:
plot(angle_amplitude,ss_JuMP,label = "JuMP", xlims = (0, 65), title = "Power Flow Convergence")
plot!(angle_amplitude, ss_newt, label = "Newton", xlabel = "Voltage Amplitude", ylabel = "Convergence (%)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 Power Flow Convergence 
 
 
 Voltage Amplitude 
 
 
 Convergence (%) 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 62.8101,23.3815 79.7388,23.3815 96.6674,47.6219 113.596,78.7883 130.525,123.806 147.453,217.305 164.382,224.231 181.311,245.009 198.239,293.49 215.168,310.804 
 232.097,335.045 249.025,335.045 265.954,355.822 282.883,348.897 299.811,348.897 316.74,355.822 333.669,355.822 350.597,345.434 367.526,355.822 384.455,366.211 
 401.383,355.822 418.312,366.211 435.241,355.822 452.169,366.211 469.098,369.674 486.027,369.674 502.955,369.674 519.884,362.748 536.813,369.674 553.741,369.674 
 570.67,369.674 587.599,366.211 604.527,369.674 621.456,359.285 638.385,366.211 655.313,362.748 672.242,369.674 689.171,369.674 706.099,369.674 723.028,369.674 
 739.957,366.211 756.885,369.674 773.814,369.674 790.743,369.674 807.671,369.674 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e9746a; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 62.8101,23.3815 79.7388,23.3815 96.6674,40.6961 113.596,75.3253 130.525,116.88 147.453,179.213 164.382,151.51 181.311,193.065 198.239,227.694 215.168,231.157 
 232.097,251.935 249.025,286.564 265.954,296.953 282.883,276.175 299.811,283.101 316.74,314.267 333.669,341.971 350.597,314.267 367.526,328.119 384.455,335.045 
 401.383,345.434 418.312,345.434 435.241,341.971 452.169,338.508 469.098,355.822 486.027,366.211 502.955,341.971 519.884,362.748 536.813,362.748 553.741,359.285 
 570.67,359.285 587.599,362.748 604.527,369.674 621.456,362.748 638.385,359.285 655.313,362.748 672.242,366.211 689.171,362.748 706.099,366.211 723.028,369.674 
 739.957,366.211 756.885,366.211 773.814,369.674 790.743,366.211 807.671,366.211 
 "/>
 
 
 
 
 JuMP 
 
 
 
 Newton

# AC Optimal Power Flow

Optimal Power Flow (OPF) is an expansion of power flow analysis in which power flow are optimized in order to minimize the cost of generation subject to the power flow constraints and other operational constraints, such as generator minimum output constraints, transmission stability and voltage constraints, and limits on switching mechanical equipment.

Equality constraints
- Power balance at each node - power flow equations

Inequality constraints
- Network operating limits (line flows, voltages)
- Limits on control variables

Solving an OPF is necessary to determine the optimal operation and planning of the grid. In this algorithm, I will simulate an optimal power flow model for a 6-bus system and determine the locational marginal prices (LMPs).